### Installing requirements


In [ ]:
%pip install semantic-kernel semantic-link

### Setting authentication parameters

In [ ]:
azure_ai_endpoint = "https://<your azure ai resource>.openai.azure.com/"
azure_ai_key = "<YOUR AZURE AI KEY>"
chat_deployment_name = "<NAME OF YOUR CHAT COMPLETION MODEL DEPLOYMENT>"
embedding_deployment_name = "<NAME OF YOUR EMBEDDING MODEL DEPLOYMENT>"
ai_skill_endpoint = "<URL OF YOUR PUBLISHED AI SKILL>"

### Setting up the kernel

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding

kernel = Kernel()

# Assigning the service to a variable first, as we'll invoke it later
chat_completion = AzureChatCompletion(
    service_id="gpt-4o",
    deployment_name=chat_deployment_name,
    endpoint=azure_ai_endpoint,
    api_key=azure_ai_key,
)
kernel.add_service(chat_completion)

kernel.add_service(
    AzureTextEmbedding(
        service_id="text-embedding",
        deployment_name=embedding_deployment_name,
        endpoint=azure_ai_endpoint,
        api_key=azure_ai_key
    ),
)

### Adding built-in plugins to the kernel

In [ ]:
from semantic_kernel.core_plugins.math_plugin import MathPlugin
from semantic_kernel.core_plugins.text_plugin import TextPlugin
from semantic_kernel.core_plugins.time_plugin import TimePlugin

kernel.add_plugin(MathPlugin(), "math")
kernel.add_plugin(TextPlugin(), "text")
kernel.add_plugin(TimePlugin(), "time")

### Adding a custom plugin to the kernel
See the Microsoft Fabric documentation at https://learn.microsoft.com/en-us/fabric/data-science/ai-skill-scenario for more details on how to call the AI Skills API.

In [ ]:
from typing import Annotated
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from synapse.ml.mlflow import get_mlflow_env_config
import requests
import json

class AISkillPlugin:

    def __init__(self, endpoint) -> None:
        self.endpoint = endpoint

    @kernel_function(
        description="Query Fabric AI Skill with a natural language question. This skill has data about the following topics: Public holidays", # Edit the description based on your data
        name="DataQuery"
    )
    def query_data(self, query: Annotated[str, "Natural language query to send to the AI Skill, should be equivalent to one SQL query"]) -> Annotated[str, "Result of the query"]:

        token = self.get_access_token()

        headers = {
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json; charset=utf-8"
        }

        question = f'{{userQuestion:"{query}"}}'

        response = requests.post(self.endpoint, headers=headers, data = question)
        response_json = json.loads(response.content)

        return response_json["result"]
    

    def get_access_token(self) -> str:
        configs = get_mlflow_env_config()
        return configs.driver_aad_token


kernel.add_plugin(
    AISkillPlugin(ai_skill_endpoint),
    "ai_skill"
)

### Adding another plugin for Semantic Link
[Semantic link](https://learn.microsoft.com/en-us/fabric/data-science/semantic-link-overview) is a feature that allows us to leverage semantic models from a notebook in Fabric. The code in this plugin is based on this [tutorial](https://learn.microsoft.com/en-us/fabric/data-science/tutorial-power-bi-measures).

In [ ]:
import sempy.fabric as fabric
from typing import List

class SemanticLinkPlugin():

    @kernel_function(
        description="List semantic models in the Microsoft Fabric workspace of the attached Lakehouse",
        name="ListDatasets"
    )
    def list_datasets(self) -> Annotated[str, "List of the names and IDs of the datasets in the workspace"]:
        return fabric.list_datasets()
        

    @kernel_function(
        description="List tables and columns in a semantic model",
        name="ListTables"
    )
    def list_tables(self, dataset: Annotated[str, "Name of the semantic model of which to list tables"]) -> Annotated[str, "List of tables in the semantic model, incl. column names"]:
        return fabric.list_tables(dataset, include_columns=True)


    @kernel_function(
        description="List measures in a semantic model",
        name="ListMeasures"
    )
    def list_measures(self, dataset: Annotated[str, "Name of the semantic model of which to list measures"]) -> Annotated[str, "List of measures in the semantic model"]:
        return fabric.list_measures(dataset)


    @kernel_function(
        description="Evaluate a measure of a semantic model to retrieve its result, before calling this function ensure you have retrieved valid groupby_columns",
        name="EvaluateMeasure"
    )
    def evaluate_measure(
        self,
        dataset: Annotated[str, "Name of the semantic model"],
        measure: Annotated[str, "Name of the measure to evaluate"], 
        groupby_columns: Annotated[List[str], 'List of columns to group by. Must be of format "Table[Column]". Use ListTables to find valid tables and columns.']
    ) -> Annotated[str, "Result of the measure"]:
        return fabric.evaluate_measure(dataset,
                measure=measure,
                groupby_columns=groupby_columns,
                filters={} # Not yet implemented
        )


kernel.add_plugin(
    SemanticLinkPlugin(),
    "semantic_link"
)


### Setting up chat history and prompt execution settings

In [ ]:
from semantic_kernel.contents import ChatHistory

chat_history = ChatHistory()
chat_history.add_system_message("You are a helpful chatbot that can assist with a variety of tasks in Microsoft Fabric. You can call AI Skills and Semantic Link to query data. Use AI Skills only when the query relates to an existing AI Skill. Use Semantic Link to find relevant datasets and measures for all otehr queries.")
chat_history.add_assistant_message("Hello! How can I help you today?")

In [ ]:
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import AzureChatPromptExecutionSettings
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

execution_settings = AzureChatPromptExecutionSettings(
    function_choice_behavior=FunctionChoiceBehavior()
)

### Conversing with the AI Assistant

In [ ]:
user_input = "check my retail analysis data for total units sold last year, group by state?" # Add your message here

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

# Get reply
chat_history.add_user_message(user_input)
reply = await chat_completion.get_chat_message_content(
    chat_history=chat_history,
    settings=execution_settings,
    kernel=kernel
)

# Update chat history
chat_history.add_assistant_message(str(reply))

In [ ]:
print(f"Assistant: {reply}")